"I'd like you to develop a visually appealing Dash application in Python. The application should be styled inline for a clean and professional appearance.

The webpage should start with a prominent heading 'Loan Application' at the top.

Following the heading, there will be a series of input boxes paired with their respective labels. Labels should be positioned to the left of the input boxes to maintain proper alignment.

    Gender: Dropdown menu with options 'Male' or 'Female'.
    Married: Dropdown menu with options 'Yes' or 'No'.
    Dependents: Dropdown menu with options '0', '1', '2', or '3+'.
    Education: Dropdown menu with options 'Graduate' or 'Not Graduate'.
    Self Employed: Dropdown menu with options 'Yes' or 'No'.
    Loan Amount: Input box with Rand currency symbol ('R') at the beginning.
    Loan Term: Input box for months, in increments of 12 for years (e.g., 12 months, 24 months, etc.).
    Credit History: Dropdown menu with options 'Yes' or 'No'; backend value to be handled as boolean.
    Property Area: Dropdown menu with options 'Urban', 'Rural', or 'Semiurban'.

Under these input fields, there should be a 'Submit' button.

Following the 'Submit' button, a message area should display feedback based on the selected inputs. For now, the message can display 'Your loan has ...' without specific content."

Added this


In [62]:


import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

from sklearn.preprocessing import StandardScaler
from category_encoders import OneHotEncoder

import joblib
from skimpy import skim


# Initialize the Dash app
app = dash.Dash(__name__)

# Define the style dictionary for consistent inline styling
input_style = {'width': '80%', 'margin': '10px'}
label_style = {'font-weight': 'bold', 'margin-right': '10px', 'display': 'inline-block', 'width': '150px'}


centered_box_style = {
    'width': '50%',
    'margin': 'auto',
    'padding': '50px',
    'border': '1px solid #ddd',
    'border-radius': '10px',
    'background-color': '#ffffff'  # Set the background color of the centered box
}

# Full page styling (to set background color outside the box)
full_page_style = {
    'background-color': '#adc5cf',  # Set the background color of the entire page
    'height': '100vh',              # Ensure the page takes full viewport height
    'display': 'flex',              # Use flexbox for vertical centering
    'justify-content': 'center',    # Center content horizontally
    'align-items': 'center'         # Center content vertically
}


app.layout = html.Div(style=full_page_style, children=[
    html.Div(style=centered_box_style, children=[
    html.H1("Loan Application", style={'text-align': 'center', 'margin-bottom': '30px'}),

    html.Div([
        html.Label("Gender:", style=label_style),
        dcc.Dropdown(
            id='gender-dropdown',
            options=[
                {'label': 'Male', 'value': 'Male'},
                {'label': 'Female', 'value': 'Female'}
            ],
            placeholder="Select gender",
            style=input_style
        )
    ]),

    html.Div([
        html.Label("Married:", style=label_style),
        dcc.Dropdown(
            id='married-dropdown',
            options=[
                {'label': 'Yes', 'value': 'Yes'},
                {'label': 'No', 'value': 'No'}
            ],
            placeholder="Marital status",
            style=input_style
        )
    ]),

    html.Div([
        html.Label("Dependents:", style={'font-weight': 'bold', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='dependents-dropdown',
            options=[
                {'label': '0', 'value': 0},
                {'label': '1', 'value': 1},
                {'label': '2', 'value': 2},
                {'label': '3+', 'value': '3+'}
            ],
            placeholder="Number of dependents",
            style={'width': '80%', 'margin': '10px'}
        )
    ]),

    html.Div([
        html.Label("Education:", style=label_style),
        dcc.Dropdown(
            id='education-dropdown',
            options=[
                {'label': 'Graduate', 'value': 'Graduate'},
                {'label': 'Not Graduate', 'value': 'Not Graduate'}
            ],
            placeholder="Education level",
            style=input_style
        )
    ]),

    html.Div([
        html.Label("Self Employed:", style=label_style),
        dcc.Dropdown(
            id='self-employed-dropdown',
            options=[
                {'label': 'Yes', 'value': 'Yes'},
                {'label': 'No', 'value': 'No'}
            ],
            placeholder="Employment status",
            style=input_style
        )
    ]),
    
        html.Div([
        html.Label("ApplicantIncome:", style=label_style),
        dcc.Input(
            id='applicant-income-input',
            type='number',
            placeholder="Enter your income",
            style=input_style
        )
    ]),
    
        html.Div([
        html.Label("Coapplicant Income:", style=label_style),
        dcc.Input(
            id='coapplicant-income-input',
            type='number',
            placeholder="Enter your coapplicants income",
            style=input_style
        )
    ]),
        

    html.Div([
        html.Label("Loan Amount (in Rands):", style=label_style),
        dcc.Input(
            id='loan-amount-input',
            type='number',
            placeholder="Enter loan amount",
            style=input_style
        )
    ]),

    html.Div([
        html.Label("Loan Term (Months):", style=label_style),
        dcc.Input(
            id='loan-term-input',
            type='number',
            placeholder="Loan term in months",
            step=12,
            style=input_style
        )
    ]),

    html.Div([
        html.Label("Credit History:", style=label_style),
        dcc.Dropdown(
            id='credit-history-dropdown',
            options=[
                {'label': 'Yes', 'value': 'Yes'},
                {'label': 'No', 'value': 'No'}
            ],
            placeholder="Credit history",
            style=input_style
        )
    ]),

    html.Div([
        html.Label("Property Area:", style=label_style),
        dcc.Dropdown(
            id='property-area-dropdown',
            options=[
                {'label': 'Urban', 'value': 'Urban'},
                {'label': 'Rural', 'value': 'Rural'},
                {'label': 'Semiurban', 'value': 'Semiurban'}
            ],
            placeholder="Select property area",
            style=input_style
        )
    ]),

    html.Button('Submit', id='submit-button', n_clicks=0, style={'margin-top': '30px'}),

    html.Div(id='output-message', style={'margin-top': '20px', 'font-size': '18px', 'font-weight': 'bold'})
])
])

def writeToFile(gender, married, dependents, education, self_employed, applicant_income, coapplicant_income,
                loan_amount, loan_term, credit_history, property_area):
    # Map 'Yes' to 1 and 'No' to 0 for Credit History
    credit_history_value = 1 if credit_history == 'Yes' else 0
    
    # Create a DataFrame with loan details
    data = {
        'Loan_ID': [1],
        'Gender': [gender],
        'Married': [married],
        'Dependents': [f"'{dependents}'"],  # Format dependents as a string with quotes
        'Education': [education],
        'Self_Employed': [self_employed],
        'ApplicantIncome': [applicant_income],
        'CoapplicantIncome': [coapplicant_income],
        'LoanAmount': [loan_amount],
        'Loan_Amount_Term': [loan_term],
        'Credit_History': [credit_history_value],
        'Property_Area': [property_area]
    }

    
    # Create DataFrame from dictionary
    df = pd.DataFrame(data)
    
    # Explicitly convert 'Dependents' column to object type (string)
    df['Dependents'] = df['Dependents'].astype(str)
    
    # Define the CSV file path
    csv_file_path = '../data/loan_application.csv'
    
    # Write DataFrame to CSV file
    df.to_csv(csv_file_path, index=False)
    print(f"File '{csv_file_path}' created successfully with loan details.")



@app.callback(
    Output('output-message', 'children'),
    [Input('submit-button', 'n_clicks')],
    [State('gender-dropdown', 'value'),
    State('married-dropdown', 'value'),
    State('dependents-dropdown', 'value'),
    State('education-dropdown', 'value'),
    State('self-employed-dropdown', 'value'),
    State('applicant-income-input', 'value'),
    State('coapplicant-income-input', 'value'),
    State('loan-amount-input', 'value'),
    State('loan-term-input', 'value'),
    State('credit-history-dropdown', 'value'),
    State('property-area-dropdown', 'value')]
)
def update_output(n_clicks, gender, married, dependents, education, self_employed, applicant_income,
                  coapplicant_income, loan_amount, loan_term, credit_history, property_area):
    if n_clicks > 0:
        writeToFile(gender, married, dependents, education, self_employed, applicant_income, coapplicant_income,
                    loan_amount, loan_term, credit_history, property_area)
        return f"Loan details submitted successfully with Load ID 1."
    else:
        return ''



# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8054)


File '../data/loan_application.csv' created successfully with loan details.
